# COMMANDE D'UN MOTEUR A COURANT CONTINU A L'AIDE D'UN TRANSISTOR MOSFET ET D'UN ESP32


## 1-Schéma de cablage du moteur à courant continu
![fritzing](image/img1.png)

In [2]:
%serialconnect --port=/dev/ttyUSB1 --baud=115200  #connexion série avec ESP32

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


On efface le main.py présent dans l'esp32, afin d'éviter les conflits.

In [3]:
import os
print("avant: ",os.listdir(""))  # affichage de la liste des fichiers contenu dans la mémoire de ESP32
os.remove('main.py')  # efface le fichier "main.py"
print("apres: ",os.listdir("")) # affichage de la liste des fichiers après 

avant:  ['boot.py', 'main.py', 'mqtt_as.py']
apres:  ['boot.py', 'mqtt_as.py']


In [3]:
%sendtofile --source mqtt_as.py

Sent 647 lines (23171 bytes) to mqtt_as.py.


On écrit dans la mémoire de ESP32 la bibliothèque mqtt_as.py et le programme principal suivant dans le fichier main.py, qui sera lancé après appui sur boot

In [4]:

%sendtofile main.py

from mqtt_as import MQTTClient, config #importation des class MQTTClient et config du module mqtt_as
from machine import Pin, PWM, ADC #importation de la classe Pin du module machine
from time import sleep #importation de la classe sleep du module time
import uasyncio#importation du module uasyncio


#configuration des connections
config['server'] = "test.mosquitto.org"#hostname du  broker
config['ssid'] = "freenab"#configuration wifi
config['wifi_pw'] = "ZimDid2023"#mot de passe

#configuration des broches D15
pwm0 = PWM(Pin(15))      # create PWM object from a pin
pwm0.freq(1000) 
#potar = Pin(34)
pwm0.duty(0)

#fonction appele lors de la communication avec le broker
def callback(topic, msg, retained):        
    if topic == b'test/pwm': 
        i=msg.decode("utf-8")        
        i=int(i)        
        pwm0.duty(i)
        
#souscription à tous les topics test/
async def conn_han(client):
    await client.subscribe('test/#', 0)
    
#programme principal  et publication sur le topic "/led"  et /D13 toutes les secondes
async def main(client):
    await client.connect() #connection du client au broker en wifi
    while True:
        #tension = ADC(potar)                          
       # tension.atten(ADC.ATTN_11DB)                        
       # await client.publish('test/potar',str(tension.read()))
       # print('potar =', tension.read())
        await uasyncio.sleep(1)

#configuration pour la gestion des publications et souscriptions
config['subs_cb'] = callback
config['connect_coro'] = conn_han

MQTTClient.DEBUG = True # affichage des messages de diagnostic
client = MQTTClient(config) #création de l'objet client par instanciation de la classe MQTTClient()

try:
    uasyncio.run(main(client)) #mise en route du programme principal 
finally:
    client.close()

Sent 49 lines (1801 bytes) to main.py.


l'ESP32 doit être connecté à via l'usb à une source d'alimentation (PC), le moteur doit aussi être connecté à une alimentation (brancher le transfo sur une prise 230V).
Vous pouvez maintenant commander la vitesse de votre moteur, pour cela il faut :
- Ouvrir VSCcodium (menu "programmation"),
- Dans VSCcodium , ouvrir le répertoire ESP32, vous verrez afficher les fichiers dans l'explorateur sur la colonne de gauche.
![fig3.png](image/fig3.png)
- ouvrir un nouveau terminal (onglet terminal sur VSCcodium)
- tapez la commande suivante : streamlit run mqtt_et_streamlit.py
 - Le navigateur s'ouvre sur une page, ou vous trouverez un slider pour compmander la vitesse du moteur
  et un graphique qui affiche la valeur de la tension présente sur l'entrée 23, on peut la faire varier en manipulant le potentiometre qui est branché sur l'entrée 23
  
